In [1]:
import pandas as pd
import glob
import os
from datetime import datetime, timedelta

In [2]:
# Find the most recent file in the directory
file_path_pattern = 'SAP_Replenishment_Reports*.xlsx'
file_list = glob.glob(file_path_pattern)

if file_list:
    file_path = file_list[0]
    print(f"Loaded file {file_path}")

    # Load the Excel file with multiple sheets 
    sheets = pd.read_excel(file_path, sheet_name=None) # Load all sheets
else:
    print("No file found matching the pattern")

Loaded file SAP_Replenishment_Reports.XLSX


In [8]:
# Find and Load the forecast file
forecast_file = glob.glob("FCST*.xlsx")[0]
forecast_data = pd.read_excel(forecast_file)

# Prepare the forecast data
# Filter only for the relevant plants
relevant_plants = ['BP01', 'BP02', 'BP04', 'BP07']
forecast_data = forecast_data[forecast_data['Plant'].isin(relevant_plants)]

# Split "Item" column to extract "Item Number"
forecast_data[['Item Number', 'Description']] = forecast_data['Item'].str.split(' - ', expand=True)

# Identify the target month columns (these names vary, so find them dynamically)
month_columns = [col for col in forecast_data.columns if 'Target' in col]

# Fill missing values in the month columns with 0 and calculate the 3-month average
forecast_data[month_columns] = forecast_data[month_columns].fillna(0)
forecast_data['AVG FCST Next 3 Months'] = forecast_data[month_columns].mean(axis=1)

# Drop duplicates while keeping the first occurence for each 'Plant' and 'Item Number'
forecast_data = forecast_data.drop_duplicates(subset=['Plant', 'Item Number'], keep='first')

# Round the AVG FCST Next 3 months values to 0 decimal places
forecast_data['AVG FCST Next 3 Months'] = forecast_data['AVG FCST Next 3 Months'].round(0)

# Keep only the columns needed for merging
forecast_data = forecast_data[['Plant', 'Item Number', 'AVG FCST Next 3 Months']]

print(forecast_data.head())

  Plant Item Number  AVG FCST Next 3 Months
0  BP02     0UC0115                     0.0
1  BP01     COV3001                   306.0
2  BP02     COV3001                     0.0
3  BP01     GN21401                  2027.0
4  BP02     GN21401                  1173.0


In [9]:
# Excluding rejected cases:
# Sales should represent succesfull transactions - Sales should reflect the actual number of cases sold and delivered to customers, not just the numbered ordered. Including rejected cases inflates the sales figures 
# and doesn't represent actual customer purchases.
# The goal of calculating average sales is typically to predict future demand or performance. Including rejected cases could be misleading to the analysis as they were not succesfully converted into sales. 

# Load and Prepare the Sales Data
sales_file = glob.glob("Rolling Weekly Sales PMI*.xlsx")[0]
sales_data = pd.read_excel(sales_file, sheet_name='Rolling Weekly no Z4, Z5, Z6 an')
sales_data = sales_data[['YearMonth', 'Plant', 'Item', 'Cases', 'Cases Reject.']]
#sales_data[['Cases', 'Cases Reject.']] = sales_data[['Cases', 'Cases Reject.']].fillna(0)
sales_data['Cases'] = sales_data['Cases'].fillna(0)
#sales_data['Net Cases'] = sales_data['Cases'] + sales_data['Cases Reject.']
sales_data['Plant'] = sales_data['Plant'].str.split(' - ').str[0]
sales_data['Item Number'] = sales_data['Item'].str.split(' - ').str[0]
sales_data = sales_data.dropna(subset=['YearMonth'])
sales_data['YearMonth'] = sales_data['YearMonth'].astype(int).astype(str).str.strip()


# Calculate the last three months dynamically 
current_date = datetime.today()
last_month = current_date.replace(day=1) - timedelta(days=1)
two_months_ago = last_month.replace(day=1) - timedelta(days=1)
three_months_ago = two_months_ago.replace(day=1) - timedelta(days=1)
last_three_months = [
    last_month.strftime('%Y%m'),
    two_months_ago.strftime('%Y%m'),
    three_months_ago.strftime('%Y%m'),
]

# Filter and calculate the average sales for the last 3 months
sales_data = sales_data[sales_data['YearMonth'].isin(last_three_months)]

# Sum Net Cases for each combination of Plant, Item Number, and YearMonth
# monthly_sales_data = sales_data.groupby(['Plant', 'Item Number', 'YearMonth'])['Net Cases'].sum().reset_index()
monthly_sales_data = sales_data.groupby(['Plant', 'Item Number', 'YearMonth'])['Cases'].sum().reset_index()

# Create a template DataFrame for the last three months to ensure every Plant and Item Number has three entries
all_combinations = pd.DataFrame(
    [(plant, item, month) for plant in monthly_sales_data['Plant'].unique()
                            for item in monthly_sales_data['Item Number'].unique()
                            for month in last_three_months],
    columns=['Plant', 'Item Number', 'YearMonth']
)

# Ensure 'YearMonth' in all_combinations is also a string
all_combinations['YearMonth'] = all_combinations['YearMonth'].astype(str)

# Merge the monthly_sales_data with this template to ensure all months are present 
# Any missing combinations will have NaN in 'Net Cases', which we will fill with 0 
monthly_sales_data = all_combinations.merge(monthly_sales_data, on=['Plant', 'Item Number', 'YearMonth'], how='left')

monthly_sales_data['Cases'] = monthly_sales_data['Cases'].fillna(0)
# monthly_sales_data['Net Cases'] = monthly_sales_data['Net Cases'].fillna(0)
# Calculate the average explicitly dividing by 3 for all entries
# avg_sales_data = monthly_sales_data.groupby(['Plant', 'Item Number'])['Net Cases'].sum().div(3).reset_index()
avg_sales_data = monthly_sales_data.groupby(['Plant', 'Item Number'])['Cases'].sum().div(3).reset_index()
# Round Net Cases
#avg_sales_data['Net Cases'] = avg_sales_data['Net Cases'].round(0)
avg_sales_data['Cases'] = avg_sales_data['Cases'].round(0)
# Change name to AVG Sales Last 3 Months
#avg_sales_data.rename(columns={'Net Cases': 'AVG Sales Last 3 Months'}, inplace=True)
avg_sales_data.rename(columns={'Cases': 'AVG Sales Last 3 Months'}, inplace=True)
print(avg_sales_data.head())

c:\conda\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  Plant Item Number  AVG Sales Last 3 Months
0  BP01     0FW0176                      0.0
1  BP01     0FZT177                      0.0
2  BP01     0UC0115                      0.0
3  BP01     0UN2107                      0.0
4  BP01     COV3001                    439.0


In [10]:
# Function to clean sheet
def clean_sheet(df):
    # Find the last ETA Week Column and delete columns after it
    eta_columns = [col for col in df.columns if col.startswith('ETA Week:')]
    if eta_columns:
        last_eta_col = eta_columns[-1] # The last ETA Week Column
        last_eta_index = df.columns.get_loc(last_eta_col) # Returns the index position of the column named last_eta_col
        df = df.iloc[:, :last_eta_index + 1] # Keep columns up to and including the last ETA Week 

    # Remove rows where all columns from 'Qty' through last ETA Week are zeros
    qty_index = df.columns.get_loc('Qty')
    cols_to_check = df.columns[qty_index:last_eta_index + 1]
    df = df[~(df[cols_to_check] == 0).all(axis=1)] # ~ excludes rows where all values in cols_to_check are zero

    # Modify ETA Week headers (remove YYYY)
    df.columns = [col.split('/')[0] if col.startswith('ETA Week:') else col for col in df.columns]

    return df

# Apply cleaning to each sheet and store the cleaned sheets
#cleaned_sheets = {sheet_name: clean_sheet(sheet_data) for sheet_name, sheet_data in sheets.items()}

In [11]:
# Merge forecast data into each warehouse tab
for sheet_name, df in sheets.items():
    ## Clean the DataFrame using the defined function
    df = clean_sheet(df)

    # Merge with the forecast data based on 'Plant' and 'Item Number'
    df = df.merge(forecast_data, on=['Plant', 'Item Number'], how='left')
    # Fill missing values in the 'AVG FCST Next 3 Months' column with 0
    df['AVG FCST Next 3 Months'] = df['AVG FCST Next 3 Months'].fillna(0)

    # Merge with sales data
    df = df.merge(avg_sales_data, on=['Plant', 'Item Number'], how='left')
    df['AVG Sales Last 3 Months'] = df['AVG Sales Last 3 Months'].fillna(0)

    # Reorginize columns: move 'AVG FCST Next 3 Months' after 'Available Stock'
    available_stock_index = df.columns.get_loc('Available Stock')
    avg_sales_col = df.pop('AVG Sales Last 3 Months')
    avg_fcst_col = df.pop('AVG FCST Next 3 Months')
    df.insert(available_stock_index + 1, 'AVG Sales Last 3 Months', avg_sales_col)
    df.insert(available_stock_index + 2, 'AVG FCST Next 3 Months', avg_fcst_col)

    # Update the sheets dictionary with the modified DataFrame
    sheets[sheet_name] = df

  

KeyError: 'AVG FCST Next 3 Months'

In [ ]:
# Saving the cleaned data back to Excel if needed
output_file_path = 'cleaned_stock_by_warehouse.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    for sheet_name, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
forecast_data_SKU = forecast_data[forecast_data['Item Number'] == 'VSS1002']
forecast_data_SKU

,Plant,Item Number,AVG FCST Next 3 Months
63,BP01,VSS1002,145.0
130,BP02,VSS1002,3.0


In [ ]:
avg_sales_data_SKU = avg_sales_data[avg_sales_data['Item Number'] == 'VSS1002']
avg_sales_data_SKU

,Plant,Item Number,AVG Sales Last 3 Months
93,BP01,VSS1002,24.0
195,BP02,VSS1002,25.0
297,BP03,VSS1002,120.0
399,BP04,VSS1002,0.0
501,BP07,VSS1002,0.0


In [ ]:
# REPORT FORMATTING 

import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, PatternFill
from datetime import date

In [ ]:
# Path to the Excel File
input_path = r'c:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Inventory\Weekly Stock Report\cleaned_stock_by_warehouse.xlsx'
output_path = r'c:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Inventory\Weekly Stock Report\formatted_report.xlsx'

# Open the Excel Workbook
wb = openpyxl.load_workbook(input_path)

# Warehouse sheets to process
warehouse_sheets = ["PSS", "PRISM", "CASESTACK", "ES3"]

# Define color 
dark_blue_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
dark_red_fill = PatternFill(start_color="963634", end_color="963634", fill_type="solid")
bright_blue_fill = PatternFill(start_color="0099FF", end_color="0099FF", fill_type="solid")
white_font = Font(color="FFFFFF")
aptos_narrow_12 = Font(name="Aptos Narrow", size=12, bold=True, color="FFFFFF")

for sheet_name in warehouse_sheets:
    if sheet_name in wb.sheetnames:
        ws = wb[sheet_name]

        # Step 1: Insert two rows at the top
        ws.insert_rows(1,2)

        # Step 2: Add Warehouse name and date in A1
        ws["A1"] = f'{sheet_name} - {date.today().strftime('%m/%d/%Y')}'
        ws.merge_cells("A1:C2")
        ws["A1"].alignment = Alignment(horizontal="center", vertical="center")
        ws["A1"].font = Font(name="Aptos Narrow", size=18, bold=True, color="FFFFFF")
        ws["A1"].fill = dark_blue_fill

        # Step 3: Label "CASES" above Item Number (D2)
        ws["D2"] = "CASES"
        ws["D2"].font = aptos_narrow_12
        ws["D2"].fill = dark_red_fill

        # Step 4: Add subtotal formula from E2 onward
        last_col = ws.max_column
        for col in range(5, last_col + 1):
            col_letter = get_column_letter(col)
            cell = ws[f"{col_letter}2"]
            if col >= 5:
                cell.value = f"=SUBTOTAL(9,{col_letter}4:{col_letter}1000)"
            cell.font = aptos_narrow_12
            cell.fill = dark_red_fill
            
        
        # Step 5: Write "RECEIVING WEEK" in H1 
        ws["H1"] = "RECEIVING WEEK"
        ws["H1"].font = aptos_narrow_12
        ws["H1"].alignment = Alignment(horizontal="center", vertical="center")
        ws["H1"].fill = bright_blue_fill
        ws.merge_cells("H1:I1")

        # Step 6: Format headers in row 3
        for cell in ws[3]:
            cell.font = aptos_narrow_12
            cell.alignment = Alignment(wrap_text=True)
            cell.fill = dark_blue_fill
        ws.row_dimensions[3].height = 47.25

        # Step 7: Apply Filters to row 3
        ws.auto_filter.ref = f"A3:{get_column_letter(last_col)}3"

        # Step 8: Freeze panes at row 4
        ws.freeze_panes = "A4"

# Save the formatted workbook
wb.save(output_path)